# tabnet Classifier

In [1]:
import time, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split

from datetime import datetime
import tqdm

import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import f1_score

/userHome/taewon/miniconda3/envs/lgai/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data pre-processing

In [2]:
# load Data
df = pd.read_csv('./open/train.csv')
df.head()

,id,epitope_seq,antigen_seq,antigen_code,start_position,end_position,number_of_tested,number_of_responses,assay_method_technique,assay_group,disease_type,disease_state,reference_date,reference_journal,reference_title,reference_IRI,qualitative_label,label
0,200001,KGILSN,AFKGILSNADIKAAEAACFKEGSFDEDGFYAKVGLDAFSADELKKL...,P02622.1,3.0,8.0,NaN,NaN,antigen inhibition,qualitative binding,Occurrence of allergy,allergic disease,1976,Int Arch Allergy Appl Immunol,The allergenic structure of allergen M from co...,http://www.iedb.org/reference/1005599,Positive,1
1,200002,SNADIK,AFKGILSNADIKAAEAACFKEGSFDEDGFYAKVGLDAFSADELKKL...,P02622.1,7.0,12.0,NaN,NaN,antigen inhibition,qualitative binding,Occurrence of allergy,allergic disease,1976,Int Arch Allergy Appl Immunol,The allergenic structure of allergen M from co...,http://www.iedb.org/reference/1005599,Positive,1
2,200003,EGSFDEDGFYAKVGLDAFSADELK,AFKGILSNADIKAAEAACFKEGSFDEDGFYAKVGLDAFSADELKKL...,P02622.1,21.0,44.0,NaN,NaN,antigen inhibition,qualitative binding,Occurrence of allergy,allergic disease,1976,Int Arch Allergy Appl Immunol,The allergenic structure of allergen M from co...,http://www.iedb.org/reference/1005599,Positive,1
3,200004,SFDEDGFY,AFKGILSNADIKAAEAACFKEGSFDEDGFYAKVGLDAFSADELKKL...,P02622.1,23.0,30.0,NaN,NaN,antigen inhibition,qualitative binding,Occurrence of allergy,allergic disease,1976,Int Arch Allergy Appl Immunol,The allergenic structure of allergen M from co...,http://www.iedb.org/reference/1005599,Positive,1
4,200005,DEDGFY,AFKGILSNADIKAAEAACFKEGSFDEDGFYAKVGLDAFSADELKKL...,P02622.1,25.0,30.0,NaN,NaN,antigen inhibition,qualitative binding,Occurrence of allergy,allergic disease,1976,Int Arch Allergy Appl Immunol,The allergenic structure of allergen M from co...,http://www.iedb.org/reference/1005599,Positive,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190811 entries, 0 to 190810
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      190811 non-null  int64  
 1   epitope_seq             190811 non-null  object 
 2   antigen_seq             190811 non-null  object 
 3   antigen_code            190811 non-null  object 
 4   start_position          190811 non-null  float64
 5   end_position            190811 non-null  float64
 6   number_of_tested        39198 non-null   float64
 7   number_of_responses     38353 non-null   float64
 8   assay_method_technique  190811 non-null  object 
 9   assay_group             190811 non-null  object 
 10  disease_type            190811 non-null  object 
 11  disease_state           180793 non-null  object 
 12  reference_date          190811 non-null  int64  
 13  reference_journal       190811 non-null  object 
 14  reference_title     

In [4]:
l_enc = LabelEncoder()

In [5]:
# make X data
df["disease_state"][df["disease_state"].isna()] ="unknown"
train = pd.DataFrame({'start_position': df['start_position'],
                  'end_position': df['end_position'],
                  'number_of_tested': df['number_of_tested'],
                  'number_of_responses': df['number_of_responses'],
                  'assay_method_technique': df['assay_method_technique'],
                  'disease_type': df['disease_type'],
                  'disease_state': df['disease_state'],
                  'target' : df['label']
                 })
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190811 entries, 0 to 190810
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   start_position          190811 non-null  float64
 1   antigen_code            190811 non-null  object 
 2   end_position            190811 non-null  float64
 3   number_of_tested        39198 non-null   float64
 4   number_of_responses     38353 non-null   float64
 5   assay_method_technique  190811 non-null  object 
 6   disease_type            190811 non-null  object 
 7   disease_state           190811 non-null  object 
 8   target                  190811 non-null  int64  
dtypes: float64(4), int64(1), object(4)
memory usage: 13.1+ MB


/tmp/ipykernel_801426/2917144474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["disease_state"][df["disease_state"].isna()] ="unknown"


In [6]:
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid"], p =[.8, .2,], size=(train.shape[0],))

train_indices = train[train.Set=="train"].index
valid_indices = train[train.Set=="valid"].index

nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object':
        print(col, train[col].nunique())
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
unused_feat = ['Set']
features = [ col for col in train.columns if col not in unused_feat+['target']]  # 7 target
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]



X_train = train[features].values[train_indices]
y_train = train['target'].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train['target'].values[valid_indices]

antigen_code 4562
assay_method_technique 29
disease_type 14
disease_state 218
Set 2


In [7]:
#X['disease_state'][X['disease_state'] == 0] ="unknown"

In [8]:
#     if X[X.columns[i]].dtype == 'object':
#         X[X.columns[i]] = X[X.columns[i]].astype('category')
# X.info()

In [9]:
# make y data
y = train["target"]
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 190811 entries, 0 to 190810
Series name: target
Non-Null Count   Dtype
--------------   -----
190811 non-null  int64
dtypes: int64(1)
memory usage: 1.5 MB


## Model train

In [10]:
clf = TabNetClassifier(cat_idxs=cat_idxs,  # cat_idxs: List[int] = field(default_factory=list)
                       cat_dims=cat_dims,  # cat_dims: List[int] = field(default_factory=list)
                       cat_emb_dim=10,     # int = 1
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-3),
                       scheduler_params={"step_size":50,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='sparsemax' # "sparsemax", entmax
                      )

Device used : cuda


In [11]:
class F1_Score(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True

    def __call__(self, y_true, y_score):
        score = f1_score(y_true, (y_score[:, 1]>0.5)*1,average='macro')
        return score

In [12]:
max_epochs = 200

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['f1'],
    max_epochs=max_epochs , patience=20,
    batch_size=1024, virtual_batch_size=128,
    num_workers=2,
    weights=1,
    drop_last=False,
)

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


epoch 0  | loss: 0.77655 | train_f1: 0.6057  | valid_f1: 0.60181 |  0:00:10s


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


epoch 1  | loss: 0.49118 | train_f1: 0.65837 | valid_f1: 0.65685 |  0:00:20s


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


...

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


epoch 10 | loss: 0.34637 | train_f1: 0.68543 | valid_f1: 0.68465 |  0:01:48s


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


epoch 11 | loss: 0.34078 | train_f1: 0.69254 | valid_f1: 0.69068 |  0:01:58s


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


## Submission

In [ ]:
test_df = pd.read_csv('./open/test.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120944 entries, 0 to 120943
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      120944 non-null  int64  
 1   epitope_seq             120944 non-null  object 
 2   antigen_seq             120944 non-null  object 
 3   antigen_code            120944 non-null  object 
 4   start_position          120944 non-null  float64
 5   end_position            120944 non-null  float64
 6   number_of_tested        42688 non-null   float64
 7   number_of_responses     42551 non-null   float64
 8   assay_method_technique  120944 non-null  object 
 9   assay_group             120944 non-null  object 
 10  disease_type            120944 non-null  object 
 11  disease_state           116281 non-null  object 
 12  reference_date          120944 non-null  int64  
 13  reference_journal       120938 non-null  object 
 14  reference_title     

In [ ]:
test_df['disease_state'][test_df['disease_state'].isna()] = "unknown"
test_df['disease_state'][test_df['disease_state']== 0 ] = "unknown"

sub_X = pd.DataFrame({'start_position': test_df['start_position'],
                  'end_position': test_df['end_position'],
                  'number_of_tested': test_df['number_of_tested'],
                  'number_of_responses': test_df['number_of_responses'],
                  'assay_method_technique': test_df['assay_method_technique'],
                  'disease_type': test_df['disease_type'],
                  'disease_state': test_df['disease_state'],
                 })
sub_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120944 entries, 0 to 120943
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   start_position          120944 non-null  float64
 1   antigen_code            120944 non-null  object 
 2   end_position            120944 non-null  float64
 3   number_of_tested        42688 non-null   float64
 4   number_of_responses     42551 non-null   float64
 5   assay_method_technique  120944 non-null  object 
 6   disease_type            120944 non-null  object 
 7   disease_state           120944 non-null  object 
dtypes: float64(4), object(4)
memory usage: 7.4+ MB


/tmp/ipykernel_737553/2040108111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['disease_state'][test_df['disease_state'].isna()] = "unknown"
/tmp/ipykernel_737553/2040108111.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['disease_state'][test_df['disease_state']== 0 ] = "unknown"


In [ ]:
nunique = sub_X.nunique()
types = sub_X.dtypes


categorical_columns = []
categorical_dims =  {}
for col in sub_X.columns:
    if types[col] == 'object':
        print(col, sub_X[col].nunique())
        sub_X[col] = l_enc.fit_transform(sub_X[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        sub_X.fillna(sub_X.loc[:, col].mean(), inplace=True)


unused_feat = ['Set']
features = [ col for col in sub_X.columns if col not in unused_feat+['target']]  # 7 target
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

sub_X = sub_X[features].values

antigen_code 2952
assay_method_technique 24
disease_type 14
disease_state 87


In [ ]:
sub_X

array([[2.548e+03, 2.930e+03, 2.563e+03, ..., 1.700e+01, 2.000e+00,
        4.100e+01],
       [5.520e+02, 2.862e+03, 5.660e+02, ..., 2.000e+00, 1.000e+01,
        1.000e+00],
       [1.560e+02, 1.745e+03, 1.670e+02, ..., 1.700e+01, 1.000e+01,
        7.100e+01],
       ...,
       [4.452e+03, 2.141e+03, 4.467e+03, ..., 1.700e+01, 2.000e+00,
        4.100e+01],
       [2.970e+02, 2.856e+03, 3.110e+02, ..., 2.000e+00, 1.000e+01,
        1.000e+00],
       [3.830e+02, 8.120e+02, 3.940e+02, ..., 1.700e+01, 2.000e+00,
        4.100e+01]])

In [ ]:
preds = clf.predict_proba(sub_X)


In [ ]:
id_list = [i for i in range(1, len(preds)+1)]
len(id_list)

120944

In [ ]:
preds

array([[1.9476056e-01, 8.0523950e-01],
       [1.0281158e-10, 1.0000000e+00],
       [2.0609664e-30, 1.0000000e+00],
       ...,
       [9.9999976e-01, 1.8575156e-07],
       [2.8117551e-02, 9.7188246e-01],
       [2.5911896e-02, 9.7408807e-01]], dtype=float32)

In [ ]:
last_preds = (preds[:, 1]>0.5)*1


In [ ]:
answer = pd.DataFrame({'id': id_list,
                      'label': last_preds
                     })
# answer.to_csv('/userHome/taekyung/song/Daycon/submission.csv', index = None) #csv파일로 생성
answer.to_csv('./submit2.csv', index = None) #csv파일로 생성